In [385]:
import pandas as pd
import numpy as np
import dedupe
import json
from random import randint, sample
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from itertools import combinations
from fuzzywuzzy import process, fuzz
from cytoolz import *

In [195]:
all_products = [p.split(',') for p in open('data/product_urls.csv', 'r').readlines()]

In [196]:
offers = [list(set([urlparse(url).path.replace('-', ' ').replace('/', ' ').replace('\n', '') for url in urls])) for urls in all_products]

In [197]:
def combine_same_product(offer_list):
    comb = list(combinations(offer_list, 2))
    return [{'offer1': c[0], 'offer2': c[1], 'label': 1} for c in comb]

In [218]:
def combine_diff_product(offer_list):
    comb = list(combinations(offer_list, 2))
    return [{'offer1': c[0], 'offer2': c[1], 'label': 0} for c in comb]

In [219]:
spc = [combine_same_product(p) for p in offers]
random_diff_offers = [p[randint(0, len(p)-1)] for p in offers]
dpc = sample(combine_diff_product(random_diff_offers), 400)

In [229]:
all_combs = []
for comb in spc:
    for p in comb:
        all_combs.append(p)
        
for comb in dpc:
    all_combs.append(comb)

In [230]:
X = []
y = []
for item in all_combs:
    X.append([item['offer1'], item['offer2']])
    y.append(item['label'])

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [238]:
vec = TfidfVectorizer(tokenizer=word_tokenize, lowercase=True)
vecs = [vec.fit_transform(x) for x in X_train]

In [244]:
terms = []
for row in offers:
    for item in row:
        terms.append(item)

In [245]:
tdidf = TfidfVectorizer(tokenizer=word_tokenize, lowercase=True)

In [246]:
tdidf.fit_transform(terms)

<392x1543 sparse matrix of type '<class 'numpy.float64'>'
	with 4957 stored elements in Compressed Sparse Row format>

In [282]:
for i in range(len(X_train)):
    f = fuzz.token_sort_ratio(*X_train[i])
    if y_train[i] == 1:
        print("offer1: {}\n offer2: {}\n similarity: {}\n".format(X_train[i][0], X_train[i][1], f))

offer1:  BelezaSaude cuidadosfemininos modelaroreseescovasrotativas Modelador De Cachos Relaxbeauty Ana Hickmann   Titanium Control Ceramica 90 A 220 C 8478010.html
 offer2:  modelador de cachos relaxbeauty ana hickmann titanium control ceramica 90 a 220 c relaxbeauty p 8530850 cp pdpb 
 similarity: 72

offer1:  TelefoneseCelulares Smartphones Android smartphone samsung galaxy s8 plus preto com 64gb tela 6 2 android 7 0 4g reconhecimento de iris camera 12mp dual pixel e processador octa core 11482056.html
 offer2:  smartphone samsung galaxy s8 sm g955fd prata dual chip android 7 0 4g wi fi com tela infinita de 62 5452395 pr
 similarity: 54

offer1:  produto 121285989 fone de ouvido philips preto shl3060bk 00
 offer2:  fone de ouvido shl3060 philips p 2119827 ea eafn 
 similarity: 65

offer1:  Informatica AcessoriosePerifericos Mouses Microsoft Mouse Optico 1850 sem Fio U7Z 00038 Vermelho 4942033.html
 offer2:  produto 120648899 mouse microsoft 1850 wireless rosa
 similarity: 37

offer1

In [293]:
with open('./meta_results.json', 'r') as f:
    m = json.loads(f.read())

In [295]:
products = m['products']

In [297]:
prs = [p['meta'] for p in products]

In [359]:
fields = [{'field': 'title', 'type': 'String', 'has missing': True},
         {'field': 'twitter:title', 'type': 'String', 'has missing': True},
         {'field': 'og:title', 'type': 'String', 'has missing': True},
         {'field': 'description', 'type': 'Text', 'has missing': True},
         {'field': 'twitter:description', 'type': 'Text', 'has missing': True},
         {'field': 'og:description', 'type': 'Text', 'has missing': True}]

In [413]:
def is_expected(item):
    expected = ['title', 'twitter:title', 'og:title', 'description', 'twitter:description', 'og:description']
    return item in expected  

def normalize_dict(d):
    fields = ['title', 'twitter:title', 'og:title', 'description', 'twitter:description', 'og:description']
    new = d
    for f in fields:
        if not new.get(f):
            d[f] = None
    return new

In [414]:
filtered = [normalize_dict(keyfilter(is_expected, merge(p))) for p in prs]

In [415]:
linker = dedupe.RecordLink(fields)

In [416]:
f1 = {'f1'+str(i): filtered[i] for i in range(len(filtered[:218]))}
f2 = {'f2'+str(i): filtered[i] for i in range(len(filtered[219:]))}

In [417]:
linker.sample(f1, f2)

In [419]:
dedupe.consoleLabel(linker)

title : None
twitter:title : None
og:title : None
description : computador all in one dell inspiron 3459-a10 intel core i3 em oferta no shoptime compre agora pelo menor preco
twitter:description : None
og:description : None

title : None
twitter:title : None
og:title : None
description : computador all in one dell inspiron 3459-a10 intel core i3 em oferta na americanas.com compre agora pelo menor preco
twitter:description : None
og:description : None

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


title : None
twitter:title : None
og:title : None
description : notebook samsung odyssey intel core i5 8gb geforce gtx 1050 em oferta na americanas.com compre agora pelo menor preco
twitter:description : None
og:description : None

title : None
twitter:title : None
og:title : None
description : notebook samsung odyssey intel core i5 8gb geforce gtx 1050 em oferta no shoptime compre agora pelo menor preco
twitter:description : None
og:description : None

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


title : None
twitter:title : geladeira/refrigerador brastemp frost free inox inverse 422l ative bre50nkana - geladeira/ refrigerador inox - magazine luiza
og:title : geladeira/refrigerador brastemp frost free inox inverse 422l ative bre50nkana - geladeira/ refrigerador inox - magazine luiza
description : geladeira/refrigerador brastemp frost free inox com as melhores condicoes voce encontra no site do magazine luiza. confira
twitter:description : geladeira/refrigerador brastemp frost free inox com as melhores condicoes voce encontra no site do magazine luiza. confira
og:description : geladeira/refrigerador brastemp frost free inox com as melhores condicoes voce encontra no site do magazine luiza. confira

title : None
twitter:title : freezer vertical consul 231l cvu26ebana - freezer vertical - magazine luiza
og:title : freezer vertical consul 231l cvu26ebana - freezer vertical - magazine luiza
description : freezer vertical consul 231l com as melhores condicoes voce encontra no site do

...

title : None
twitter:title : None
og:title : og-title
description : encontre aqui sanduicheira minigrill colors com chapa antiaderente san231 vermelha - cadence com otimos precos na ricardoeletro.com. confira
twitter:description : None
og:description : og-description

title : None
twitter:title : None
og:title : og-title
description : encontre aqui perfume lancome la vie est belle feminino eau de parfum 30ml com otimos precos na ricardoeletro.com. confira
twitter:description : None
og:description : og-description

10/10 positive, 9/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


title : None
twitter:title : jogo forza horizon 3 - xbox one
og:title : jogo forza horizon 3 - xbox one  casasbahia.com.br
description : jogo forza horizon 3 com o menor preco voce so encontra aqui na loja microsoft. 
twitter:description : jogo forza horizon 3 com o menor preco voce so encontra aqui na loja microsoft. 
og:description : None

title : None
twitter:title : None
og:title : forza horizon 3 - xbox one
description : - este e o seu horizon - voce esta no comando do festival horizon. personalize tudo, contrate e demita seus amigos e explore a australia em mais de 350 dos melhores carros do mundo. torne seu horizon a celebracao suprema de carros, musica e liberdade nas 
twitter:description : None
og:description : este e o seu horizon - voce esta no comando do festival horizon. personalize tudo, contrate e demita seus amigos e explore a australia em mais de 350 dos melhores carros do mundo. torne seu horizon a celebracao suprema de carros, musica e liberdade nas estradas. como ch

f


Finished labeling


In [420]:
linker.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.001000
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.8, og:title)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.6, og:title)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.4, og:title)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.2, og:title)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.8, og:title)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.2, og:title)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.6, og:title)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.4, og:title)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (3, og:title)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (1, og:title)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (4, og:title)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (2, og:title)
INFO:dedupe.blocking:C

INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.2, title)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.6, title)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.8, title)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.6, og:description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.4, og:description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.2, og:description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.8, og:description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (1, og:description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (2, og:description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (4, og:description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (3, og:description)
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(ExistsPredicate: (Exists, description), SimplePredicate: (commonThreeTokens, og:title))
INFO:dedupe.training:(Simple

In [422]:
matchs = linker.match(f1,f2)

INFO:dedupe.api:0 records
INFO:dedupe.api:100 records
INFO:dedupe.api:200 records


In [423]:
matchs

[(('f10', 'f20'), 1.0), (('f11', 'f21'), 1.0), (('f12', 'f22'), 1.0), (('f13', 'f23'), 1.0), (('f14', 'f24'), 1.0), (('f15', 'f25'), 1.0), (('f16', 'f26'), 1.0), (('f17', 'f27'), 1.0), (('f18', 'f28'), 1.0), (('f111', 'f211'), 1.0), (('f112', 'f212'), 1.0), (('f113', 'f213'), 1.0), (('f114', 'f214'), 1.0), (('f115', 'f215'), 1.0), (('f116', 'f216'), 1.0), (('f117', 'f217'), 1.0), (('f118', 'f218'), 1.0), (('f119', 'f219'), 1.0), (('f120', 'f220'), 1.0), (('f121', 'f221'), 1.0), (('f122', 'f222'), 1.0), (('f123', 'f223'), 1.0), (('f124', 'f224'), 1.0), (('f126', 'f226'), 1.0), (('f128', 'f228'), 1.0), (('f129', 'f229'), 1.0), (('f130', 'f230'), 1.0), (('f131', 'f231'), 1.0), (('f132', 'f232'), 1.0), (('f133', 'f233'), 1.0), (('f134', 'f234'), 1.0), (('f135', 'f235'), 1.0), (('f136', 'f236'), 1.0), (('f137', 'f237'), 1.0), (('f138', 'f238'), 1.0), (('f140', 'f240'), 1.0), (('f141', 'f241'), 1.0), (('f144', 'f244'), 1.0), (('f145', 'f245'), 1.0), (('f146', 'f246'), 1.0), (('f147', 'f247')

In [424]:
for i in matchs:
    print('offer1: {}\n offer2: {}\n'.format(f1[i[0][0]], f2[i[0][1]]))

offer1: {'twitter:title': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston ', 'twitter:description': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston  - memorias no casasbahia.com.br', 'og:title': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston   casasbahia.com.br', 'description': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston ', 'title': None, 'og:description': None}
 offer2: {'twitter:title': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston ', 'twitter:description': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston  - memorias no casasbahia.com.br', 'og:title': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston   casasbahia.com.br', 'description': 'memoria 4gb 1x4gb ddr3 1600mhz hyperx fury red hx316c10fr/4 kingston ', 'title': None, 'og:description': None}

offer1: {'twitter:title': 'memoria 16gb 2x8gb ddr3 1600mhz hyperx fury black hx316